# Prétraitement du texte

Le texte libre n'est jamais simple. A chaque environnement son langage. On n'écrit pas sur un réseau social comme dans un dictionnaire. Comment convertir cela en numérique ? That is the question.


## Bog of Words

C'est le début de tout. La première étape consiste à découper un texte en token (caractères, mots, ...). Le plus souvent, c'est en mot. Chaque mot reçoit un identifiant. Une phrase est transformée en une liste d'entiers.

L'approche la plus simple consiste ensuite un vecteur binaire pour chaque mot $(v_1, ..., v_i, ..., v_n)$. $n$ est le nombre de mots reconnus par le modèle. $v_i in \{ 0, 1 \}$, il vaut 1 si i est égale à son numéro, 0 sinon. Chaque vecteur ne contient qu'un seul un.

On fait ensuite la somme pour obtenir un seul vecteur par phrase, quelle que soit la longueur de la phrase : [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    "J'adore le machine learning et l'intelligence artificielle.",
    "Les réseaux de neurones sont puissants pour la vision par ordinateur.",
    "Scikit-learn est une super bibliothèque pour le ML.",
    "Transformer est un modèle de deep learning très utilisé.",
]

vectorizer = CountVectorizer(max_features=10)
X = vectorizer.fit_transform(corpus)

print("Shape de la matrice TF-IDF :", X.shape)

Shape de la matrice TF-IDF : (4, 10)


In [10]:
X.todense()

matrix([[1, 0, 0, 0, 0, 1, 1, 1, 1, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 1],
        [0, 1, 0, 0, 1, 0, 0, 1, 0, 1],
        [0, 0, 1, 1, 1, 0, 0, 0, 1, 0]])

0 ou 1, peut-on faire mieux ? C'est l'objectif de l'approche [TF-IDF](https://fr.wikipedia.org/wiki/TF-IDF), donner un poids à chaque mot qui dépend de sa fréquence dans le document, et de sa fréquence dans l'ensemble des documents. On souhaite éliminer les mots rares, trop rares pour être significatifs, ou trop fréquent comme les [stop-words](https://fr.wikipedia.org/wiki/Mot_vide), si fréquents que les enlever n'enlève au sens de la phrase : [TfIdfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfIdfVectorizer.html).

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
    "J'adore le machine learning et l'intelligence artificielle.",
    "Les réseaux de neurones sont puissants pour la vision par ordinateur.",
    "Scikit-learn est une super bibliothèque pour le ML.",
    "Transformer est un modèle de deep learning très utilisé.",
]

vectorizer = TfidfVectorizer(max_features=10)
X = vectorizer.fit_transform(corpus)

print("Shape de la matrice TF-IDF :", X.shape)
X.todense()

Shape de la matrice TF-IDF : (4, 10)


matrix([[0.48546061, 0.        , 0.        , 0.        , 0.        ,
         0.48546061, 0.48546061, 0.38274272, 0.38274272, 0.        ],
        [0.        , 0.        , 0.70710678, 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.70710678],
        [0.        , 0.59081908, 0.        , 0.        , 0.46580855,
         0.        , 0.        , 0.46580855, 0.        , 0.46580855],
        [0.        , 0.        , 0.46580855, 0.59081908, 0.46580855,
         0.        , 0.        , 0.        , 0.46580855, 0.        ]])

In [15]:
vectorizer.get_feature_names_out()

array(['artificielle', 'bibliothèque', 'de', 'deep', 'est', 'et',
       'intelligence', 'le', 'learning', 'pour'], dtype=object)

## n-grammes

Ces deux approches ne donne aucun poids à l'ordre des mots. Changer l'ordre ne change rien. Pour le remettre, on utilise des n-grammes, cela revient à considérer les mots, les couples de mots, les triplets de mots...

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
    "J'adore le machine learning et l'intelligence artificielle.",
    "Les réseaux de neurones sont puissants pour la vision par ordinateur.",
    "Scikit-learn est une super bibliothèque pour le ML.",
    "Transformer est un modèle de deep learning très utilisé.",
]

vectorizer = TfidfVectorizer(max_features=10, ngram_range=(1, 2))
X = vectorizer.fit_transform(corpus)

print("Shape de la matrice TF-IDF :", X.shape)
X.todense()

Shape de la matrice TF-IDF : (4, 10)


matrix([[0.66767854, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.52640543, 0.52640543, 0.        ],
        [0.        , 0.        , 0.        , 0.52640543, 0.66767854,
         0.        , 0.        , 0.        , 0.        , 0.52640543],
        [0.        , 0.50867187, 0.50867187, 0.        , 0.        ,
         0.        , 0.40104275, 0.40104275, 0.        , 0.40104275],
        [0.        , 0.        , 0.        , 0.46580855, 0.        ,
         0.59081908, 0.46580855, 0.        , 0.46580855, 0.        ]])

In [17]:
vectorizer.get_feature_names_out()

array(['artificielle', 'bibliothèque', 'bibliothèque pour', 'de',
       'de neurones', 'deep', 'est', 'le', 'learning', 'pour'],
      dtype=object)

## Deep Learning

Au final, il s'agit de compresser des phrases dans un espace vectoriel numérique. Plus on a de texte, plus on peut apprendre des compressions efficaces. Le deep learning, la puissance de calcul vient à la rescousse. Une approche populaire est [word2vec](https://towardsdatascience.com/word2vec-with-pytorch-implementing-original-paper-2cd7040120b0/). Un autre package [textblob](https://textblob.readthedocs.io/en/dev/) propose d'enrichir les phrases en taggant les mots (nom, verbe, ...). Il y a aussi [spacy](https://spacy.io/), [NLTK](https://www.nltk.org/).

Le plus efficace est sans doute d'utiliser un modèle de deep learning entraîné à faire une tâche proche du problème de prédiction à résoudre.

In [20]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel

# Charger le tokenizer et le modèle
MODEL_NAME = "google/bert_uncased_L-2_H-128_A-2"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

config.json:   0%|          | 0.00/382 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

In [21]:
def get_embedding(text):
    """Convertir un texte en embedding."""
    tokens = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        output = model(**tokens)
    return output.last_hidden_state[:, 0, :].numpy()  # Prendre le CLS token


# Appliquer à notre dataset
X_bert = np.vstack([get_embedding(t) for t in corpus])
print("Shape des embeddings BERT :", X_bert.shape)  # (5, 768)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Shape des embeddings BERT : (4, 128)


In [22]:
X_bert

array([[-1.01954079e+00,  1.09865868e+00, -3.24030900e+00,
        -1.63258791e+00, -1.05619824e+00, -1.61958218e-01,
         5.81856966e-01,  1.30005205e+00, -8.13747764e-01,
        -1.70176730e-01,  9.68299985e-01, -1.26105383e-01,
         1.74509242e-01, -1.15167648e-01,  1.64889967e+00,
        -5.84569156e-01,  7.68755794e-01,  1.78837568e-01,
        -1.32599080e+00,  1.69023693e-01, -1.79380298e-01,
        -5.71163356e-01,  3.29246068e+00,  6.05863452e-01,
         1.45341444e+00, -1.82031751e-01,  3.90212893e-01,
         6.18985474e-01,  6.09537959e-01, -1.06283987e+00,
        -3.49633962e-01, -4.75799978e-01, -2.02315283e+00,
         2.34588310e-01,  3.09544921e-01, -1.70815694e+00,
        -4.13940072e-01,  1.76980734e-01, -3.81380868e+00,
        -5.86692035e-01,  3.51953477e-01,  3.45019139e-02,
         1.19410813e+00, -1.84602499e+00, -1.17189325e-01,
        -1.89737916e+00, -1.35886836e+00, -1.02250898e+00,
         1.80050611e-01, -4.42315340e-01, -1.75411665e+0